In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-midterm-p4/sample.csv
/kaggle/input/2022-ml-midterm-p4/train.csv
/kaggle/input/2022-ml-midterm-p4/test.csv


In [2]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
sample = pd.read_csv('../input/2022-ml-midterm-p4/sample.csv')
test = pd.read_csv('../input/2022-ml-midterm-p4/test.csv')
train = pd.read_csv('../input/2022-ml-midterm-p4/train.csv')

In [4]:
train.head()
trainX = train.drop(['price', 'ID'], axis = 1)
trainY = train['price']
test = test.drop(['ID'], axis = 1)

In [5]:
trainX.head()

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,4,Auris,2015,1,47541,4,145,46.3,1.6
1,0,X2,2018,3,5000,0,145,50.4,2.0
2,2,Focus,2017,3,47018,4,145,51.4,1.0
3,2,Focus,2020,1,1550,4,145,49.6,1.0
4,0,3 Series,2017,3,23505,0,145,64.2,2.0


In [6]:
print(type(test['model']))

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

a = pd.concat([trainX['model'], test['model']])
le.fit(a)
trainX['model'] = le.transform(trainX['model'])
test['model'] = le.transform(test['model'])
trainX.head()

<class 'pandas.core.series.Series'>


,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,4,15,2015,1,47541,4,145,46.3,1.6
1,0,144,2018,3,5000,0,145,50.4,2.0
2,2,26,2017,3,47018,4,145,51.4,1.0
3,2,26,2020,1,1550,4,145,49.6,1.0
4,0,100,2017,3,23505,0,145,64.2,2.0


In [7]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
trainX_sc = scaler.fit_transform(trainX)
test_sc = scaler.transform(test)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainX_sc, trainY, test_size=0.33, random_state=42)

In [9]:
from sklearn.neighbors import KNeighborsRegressor
# knn regressor 호출
neigh = KNeighborsRegressor(n_neighbors=7,  p = 1, weights = 'distance')
# 학습시키기
neigh.fit(trainX_sc, trainY)
#print(neigh.score(X_test, y_test))
#예측하기
pred = neigh.predict(test_sc)

# from sklearn.linear_model import LogisticRegression
# logis = LogisticRegression()
# logis.fit(trainX_sc, trainY)
# pred = logis.predict(test_sc)

In [10]:
sample['price'] = pred
sample.to_csv('submission.csv', index = False)

분류 문제가 아닌 회귀 문제여서 knnregressor를 사용하였다. 
logistic regression으로 예측한 것보다 우수한 성능을 보였다.
하이퍼파라미터가 3이나 7일 때보다 우수한 성능을 보였다.

# Review

1. label encoder로 'model'의 값을 labeling 해주었다.
    이 때, train data에 나오지 않은 값이 test data에 있었기 때문에
    두 데이터를 concat을 이용하여 합친 다음 fit을 해주고, 각각
    transform 해주었다.
    
2. 